In [ ]:
import numpy as np
from copy import deepcopy
from graph.graph import RectangleGraph
from shipmodel.shipModel import shipModel

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.ticker as ticker
%matplotlib inline

# Add functions

In [ ]:
# Calculate estimate of arrival
def calculateEta(route, speed, uMatrix, vMatrix):
    distance = 1e5
    time = 0
    for i in range(len(route) - 1):
        cu = uMatrix[route[i+1]]
        cv = vMatrix[route[i+1]]
        if route[i][0] == route[i+1][0]:
            if route[i+1][1] - route[i][1] == 1:
                angle = 0
            else:
                angle = 180
        elif route[i+1][0] - route[i][0] == 1:
            angle = 90
        else: 
            angle = 270
        time += distance / shipModel.speedGPS2Water(speed, angle, cu, cv)
    return time            

In [ ]:
# matrix for weight function in path finding
def createMap(uMatrix, vMatrix, shape, num):
    return np.sqrt(uMatrix[:,num].reshape(shape) ** 2 + vMatrix[:,num].reshape(shape) ** 2)

# Add components

## Create graph

In [ ]:
RGraph = RectangleGraph(50, 40)

## Create random current sample
CurrentU is current amplitude on horizontal direction (longitude direction)


CurrentV is current amplitude on vertical direction (latitude direction)

In [ ]:
currentU, Zshape = RGraph.weightSPD(3000)
currentV, _ = RGraph.weightSPD(3000)

## Add obstacle to the graph 

In [ ]:
# copy graph in the instance
graph = deepcopy(RGraph.graph)
# add obstacles
obstacles = []
obstacles.append(RGraph.addObstacle(graph, (5,5), (10, 10)))    
obstacles.append(RGraph.addObstacle(graph, (10,10), (15, 15)))
obstacles.append(RGraph.addObstacle(graph, (15,15), (20, 20)))
obstacles.append(RGraph.addObstacle(graph, (20,20), (35, 35)))
obstacles.append(RGraph.addObstacle(graph, (35,35), (39, 39)))

## Generate one weight function 

In [ ]:
colorMap = createMap(currentU, currentV, Zshape, 0)

## Find optimal route based on the above weight function

In [ ]:
sampleRoute = RGraph.findShortestRoute(graph, (0,0), (34,39), colorMap)

In [ ]:
ETA = []
speed = 10
for i in range(currentU.shape[1]):
    cu = currentU[:, i].reshape(Zshape)
    cv = currentV[:, i].reshape(Zshape)
    ETA.append(calculateEta(sampleRoute, speed, cu, cv) / 3600 )    

# Drawing 

In [ ]:
@ticker.FuncFormatter
def major_formatter(x, pos):
    return "%.1f%%" % (x * 100)

fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(1,1,1)
n, bins, patches = ax.hist(
    ETA,
    25,
    density=True,
    facecolor='b', 
    alpha=0.75, 
    edgecolor='k',
    lw = 1
)
ax.set_title("ETA histogram", fontsize = 20, fontweight='bold')

ax.set_xlabel('Travel Hours', fontsize = 18, fontweight='bold')
ax.set_ylabel('Probability',fontsize = 18, fontweight='bold')
ax.tick_params(labelsize = 15)
fmt = '%.0f%%' 
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(major_formatter)
ax.grid(True, alpha = 0.5, ls='-.')
plt.show()

In [ ]:
nodelist = list(graph)
xy = np.asarray(nodelist)
edgelist = list(graph.edges())

In [ ]:
fig = plt.figure(figsize=(21,10))

ax = fig.add_subplot(1,2,1)
ax.set_title("Sample weather condition 1", fontsize = 20, fontweight='bold')
ax.scatter(
    xy[:, 0],
    xy[:, 1],
    s=5,
    c='k',
    marker='o',
)

for u, v in edgelist:
    ax.plot(
        [u[0], v[0]], 
        [u[1], v[1]], 
        color = 'k', lw = 1)
for obstacle in obstacles:
    ax.plot(
            obstacle[0], 
            obstacle[1], 
            color = 'r', lw = 3)
xy1 = np.asarray(sampleRoute)
ax.plot(xy1[:,0], xy1[:,1], color = 'b', lw=3)
ax.scatter(0, 0, s=30,  c='b', marker='o')
ax.scatter(34, 39, s=30,  c='b', marker='o')
    

ax.imshow( currentU[:,1].reshape(Zshape), origin="lower", aspect="equal" ,
          extent = ( -0.2,49.1,-0.2,39.2))
ax.tick_params(
    axis="both",
    which="both",
    bottom=False,
    left=False,
    labelbottom=False,
    labelleft=False,
)
##################

ax1 = fig.add_subplot(1,2,2)
ax1.set_title("Sample weather condition 2", fontsize = 20, fontweight='bold')
ax1.scatter(
    xy[:, 0],
    xy[:, 1],
    s=5,
    c='k',
    marker='o',
)

for u, v in edgelist:
    ax1.plot(
        [u[0], v[0]], 
        [u[1], v[1]], 
        color = 'k', lw = 1)
for obstacle in obstacles:
    ax1.plot(
            obstacle[0], 
            obstacle[1], 
            color = 'r', lw = 3)
xy1 = np.asarray(sampleRoute)
ax1.plot(xy1[:,0], xy1[:,1], color = 'b', lw=3)
ax1.scatter(0, 0, s=30,  c='b', marker='o')
ax1.scatter(34, 39, s=30,  c='b', marker='o')
    

ax1.imshow( currentU[:,2].reshape(Zshape), origin="lower", aspect="equal" ,
          extent = ( -0.2,49.1,-0.2,39.2))
ax1.tick_params(
    axis="both",
    which="both",
    bottom=False,
    left=False,
    labelbottom=False,
    labelleft=False,
)

plt.tight_layout()
plt.show()